# Stage 0: Load the Libraries

In [1]:
# Stage 0: import Libraries
import pandas
import numpy
import requests
import json
import boto3
from langchain_aws import BedrockLLM
from langchain_community.chat_models import BedrockChat

# Stage 1: AWS Configuration and Define Global Variables


In [5]:
llm = BedrockLLM(
    model_id="mistral.mistral-large-2402-v1:0"
)

In [6]:
#  Define AWS Global Variables
AWS_KB_ID = "OY4K3P2JUR"
client = boto3.client('bedrock-agent-runtime')




In [7]:
# Define Aspiration
ASPIRATION = "I want to be a SAP consultant"


In [15]:
context = client.retrieve(knowledgeBaseId=AWS_KB_ID, retrievalQuery={
        'text': ASPIRATION
    })

# Stage 2: Query the Knowledge Base and Transform the context

In [16]:
response = client.retrieve(
    knowledgeBaseId=AWS_KB_ID,
     retrievalQuery={
        'text': ASPIRATION,
    }
)




# Stage 3: Prompt Design & Data Models

In [17]:
# Stage 3: Prompt Design & Data Models

skills_extraction_prompt = """
You are Career Consultant. You are tasked with extracting skills required to reach the aspiration below.

Task:
1. Understand the Job Descriptions below relevant to the aspiration.
2. From your Undestanding, extract all the skill entities required to reach the aspiration.
3. For each skill,be specfic, provide the actual skill extracted, do not combine skills together and list them out individually.
4. Also provide justification of the reason why the skills is needed to reach the aspiration.
5. For each skill level, be strict , Beginner: 0-1 years, Intermediate: 1-3 years, Advanced: 3-5 years, Expert: 5+ years.
4. For each skill level, provide a justification of how the aspiration requires a skill of that level.


FORMAT Response Instruction:
----------------
{format_instructions}
----------------


----------------
aspiration: {aspiration}
----------------
----------------
job descriptions: {job_descriptions}
----------------



"""

fix_format_instruction = """
--------------
{instructions}
--------------
Completion:
--------------
{completion}
--------------
Above, the Completion did not satisfy the constraints given in the Instructions.
Error:
--------------
{error}
--------------
Please try again. Please only respond with an answer that satisfies the constraints laid out in the Instructions.
Important:  Only correct the structural issues within the JSON format. Do not modify the existing data values themselves:
"""

In [12]:
# Data Models
from typing import List
from pydantic import BaseModel, Field

# class Competency(BaseModel):
#     competency_type: str = Field(..., description="Type of competency:Functional, Technical")
#     competency_name: str = Field(..., description="Name of the competency")
#     competency_description: str = Field(..., description="Description of how the client has demonstrated the competency in their resume")

# class Comptencies(BaseModel):
#     competencies: List[Competency] = Field(..., description="List of competencies extracted from the resume")

class Skill(BaseModel):
    skill_justification: str = Field(..., description="Justification of the extracted skill")
    skill: str = Field(..., description="Name of the skill")
    skill_level_justification : str = Field(..., description="Justification of the skill level")    
    skill_level: str = Field(..., description="Level of the skill", enum=["Beginner", "Intermediate", "Advanced", "Expert"])

class Skills(BaseModel):
    skills: List[Skill] = Field(..., description="List of skills and their corresponding skill level needed for the aspiration")
    


# Stage 4: Extract Skills required for aspiration

In [20]:
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser
from langchain.chains import TransformChain

def fix_chain_fun(inputs):    
    fix_prompt = PromptTemplate.from_template(fix_format_instruction)
    fix_prompt_str = fix_prompt.invoke({'instructions':inputs['instructions'],
                                        'completion':inputs['completion'],
                                        'error':inputs['error']}).text
    
    #print(fix_prompt_str)
    
    completion = llm.invoke(fix_prompt_str)

    return {"completion": completion}

fix_chain = TransformChain(
    input_variables = ["instructions", "completion", "error"],output_variables=["completion"], transform=fix_chain_fun
)

def extract_skills(aspiration, job_descriptions: str) -> Skills:
    # Invoke the LLM
    parser = PydanticOutputParser(pydantic_object=Skills)
    
    fix_parser = OutputFixingParser(
        parser=parser,
        retry_chain=fix_chain,
        max_retries=2
    )

    prompt = PromptTemplate(
        template = skills_extraction_prompt, 
        input_variables=["aspiration", "job_descriptions"],
        partial_variables= {"format_instructions": parser.get_format_instructions()})
    
    prompt_str = prompt.format(aspiration=aspiration, job_descriptions=job_descriptions)

    print(prompt_str)
    
    response = llm.invoke(prompt_str)
    response = response

    print(f"Response is : {response}")

    fixed_response = fix_parser.invoke(response).dict()

    return fixed_response



In [21]:
aspiration_extracted_skills = extract_skills(ASPIRATION, context)


You are Career Consultant. You are tasked with extracting skills required to reach the aspiration below.

Task:
1. Understand the Job Descriptions below relevant to the aspiration.
2. From your Undestanding, extract all the skill entities required to reach the aspiration.
3. For each skill,be specfic, provide the actual skill extracted, do not combine skills together and list them out individually.
4. Also provide justification of the reason why the skills is needed to reach the aspiration.
5. For each skill level, be strict , Beginner: 0-1 years, Intermediate: 1-3 years, Advanced: 3-5 years, Expert: 5+ years.
4. For each skill level, provide a justification of how the aspiration requires a skill of that level.


FORMAT Response Instruction:
----------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type"

In [22]:
with open("aspiration_extracted_skills.json", "w") as f:
    json.dump(aspiration_extracted_skills, f, indent=4) 